In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')
# If needed, add directory names
os.chdir('/content/drive/My Drive/Code/EXTRA DATASET/friends-corpus')

Mounted at /content/drive


In [2]:
import pandas as pd
import os
import numpy as np
import re
import pickle
import json

In [4]:
# The utterances file containts all needed data and can be read in easily in a pandas dataframe
df = pd.read_json('utterances.json', lines = True)
df.head()

,id,conversation_id,text,speaker,meta,reply-to,timestamp
0,s01_e01_c01_u001,s01_e01_c01_u001,There's nothing to tell! He's just some guy I ...,Monica Geller,"{'tokens': [['There', ''s', 'nothing', 'to', '...",None,NaT
1,s01_e01_c01_u002,s01_e01_c01_u001,"C'mon, you're going out with the guy! There's ...",Joey Tribbiani,"{'tokens': [['C'mon', ',', 'you', ''re', 'goin...",s01_e01_c01_u001,NaT
2,s01_e01_c01_u003,s01_e01_c01_u001,"All right Joey, be nice. So does he have a hum...",Chandler Bing,"{'tokens': [['All', 'right', 'Joey', ',', 'be'...",s01_e01_c01_u002,NaT
3,s01_e01_c01_u004,s01_e01_c01_u001,"Wait, does he eat chalk?",Phoebe Buffay,"{'tokens': [['Wait', ',', 'does', 'he', 'eat',...",s01_e01_c01_u003,NaT
4,s01_e01_c01_u005,s01_e01_c01_u001,,TRANSCRIPT_NOTE,"{'tokens': [], 'character_entities': [], 'emot...",s01_e01_c01_u004,NaT


In [6]:
# Split the id column in seasons
df['Season'] = df['id'].str.split('_').str[0]
df['Episode'] = df['id'].str.split('_').str[1]
df['Season_Episode'] = df['id'].str.split('_').str[0:2]
# These columns are not needed at the moment
df = df.drop(['id','meta','reply-to','timestamp'],axis=1)
df.head()

,conversation_id,text,speaker,Season,Episode,Season_Episode
0,s01_e01_c01_u001,There's nothing to tell! He's just some guy I ...,Monica Geller,s01,e01,"[s01, e01]"
1,s01_e01_c01_u001,"C'mon, you're going out with the guy! There's ...",Joey Tribbiani,s01,e01,"[s01, e01]"
2,s01_e01_c01_u001,"All right Joey, be nice. So does he have a hum...",Chandler Bing,s01,e01,"[s01, e01]"
3,s01_e01_c01_u001,"Wait, does he eat chalk?",Phoebe Buffay,s01,e01,"[s01, e01]"
4,s01_e01_c01_u001,,TRANSCRIPT_NOTE,s01,e01,"[s01, e01]"


In [9]:
# List of all unique movie ids, assuming there for every movie id: is connected to at precisely one movie
seasons = df['Season'].unique()
episodes = df['Episode'].unique()

# Create a dictionairy with movie ids as keys
seasonal_frames = {(id,ep) : df[(df['Season']==id) & (df['Episode']==ep)] for id in seasons for ep in episodes if len(df[(df['Season']==id) & (df['Episode']==ep)]) != 0}
len(seasonal_frames)

236

In [10]:
keys = list(seasonal_frames.keys())
frames = [seasonal_frames[key].reset_index() for key in keys]
frames[0]

,index,conversation_id,text,speaker,Season,Episode,Season_Episode
0,0,s01_e01_c01_u001,There's nothing to tell! He's just some guy I ...,Monica Geller,s01,e01,"[s01, e01]"
1,1,s01_e01_c01_u001,"C'mon, you're going out with the guy! There's ...",Joey Tribbiani,s01,e01,"[s01, e01]"
2,2,s01_e01_c01_u001,"All right Joey, be nice. So does he have a hum...",Chandler Bing,s01,e01,"[s01, e01]"
3,3,s01_e01_c01_u001,"Wait, does he eat chalk?",Phoebe Buffay,s01,e01,"[s01, e01]"
4,4,s01_e01_c01_u001,,TRANSCRIPT_NOTE,s01,e01,"[s01, e01]"
...,...,...,...,...,...,...,...
318,318,s01_e01_c15_u001,"Yeah. Yeah, I'll have a cup of coffee.",#ALL#,s01,e01,"[s01, e01]"
319,319,s01_e01_c15_u001,"Kids, new dream... I'm in Las Vegas.",Chandler Bing,s01,e01,"[s01, e01]"
320,320,s01_e01_c15_u001,"Ahh, miss? More coffee?",Customer,s01,e01,"[s01, e01]"
321,321,s01_e01_c15_u001,"Ugh. Excuse me, could you give this to that gu...",Rachel Green,s01,e01,"[s01, e01]"


In [11]:
lens = [len(frame) for frame in frames]
sum(lens)

67373

In [12]:
## Function to temporary remove normal/expected characters
def remnormals(sen):
    l = sen
    l = l.replace('.','').replace('?','').replace('!','').replace(',','').replace('--','').replace('-','').replace('\n','').replace(';','')
    l = l.replace(':','').replace('\"','').replace('/','').replace('`','').replace('…','').replace("'",'').replace("´",'').replace("·",'')
    l = l.replace("«",'').replace("»",'').replace("’",'').replace("$",'').replace("+",'')#.replace(')','')
    l = l.replace("“",'').replace("”",'').replace("£",'').replace("%",'').replace('‘','')#.replace('(','').replace("[",'').replace("]",'')

    return l

In [13]:
%%time
# Check weird characters or letters
n=-1
for frame in frames:
  for sen in frame['text']:
    n+=1

    check = sen
    l = sen
    l = remnormals(sen)

    # This way checked what strange letters and were.
    e = set(re.findall('[ÃÂâãËë]',l))

    # First check was on < 3
    if len(sen) < 2 and sen != '':
      print(frame['Season'].unique(),sen)
      stoploop = 're'

    # Larger than 1 because of ' '
    if len(e) > 1:
#      print(frame['Season'].unique(),' sentence: ',sen,'\n These are the characters: ',e,'\n')
       stoploop = 're'
    # Other strange characters.
    e = set(re.findall('[^\w\*]',l))

    if len(e) > 1:
#      print(frame['Season'].unique(),' sentence: ',sen,'\n These are the characters: ',e,'\n')
       stoploop = 're'

['s03'] S
['s06'] .
CPU times: user 374 ms, sys: 1.91 ms, total: 376 ms
Wall time: 378 ms


In [14]:
# Load in speaker names
speakers = json.load(open('speakers.json'))
speakernames = set(list(speakers.keys()))
#speakernames

In [15]:
# Check if there are action instances usually indicated by capital letters speaker does x, f.e. FLORA does x
for speaker in list(speakernames):
  for frame in frames:
    for sen in frame['text']:
      if speaker in sen:
        #print(speaker,' sen',sen,'\n')
       # stoploop ='re'
        if 'LINE OF DIALOGUE CUT OFF' in sen or 'SCENE' in sen or 'DELETE' in sen or 'CONTINUED' in sen:
          print(sen)
        #    stoploop ='re'


In [16]:
n=-1
# Check brackets because of weird occurances check ~
# Some files have brackets around the sentences, some for translations and some randomly
for frame in frames:
  frame['text'] = frame.text.str.replace('</u>','').str.replace('<u>','').str.replace('</i>','').str.replace('<i>','').str.replace('<b>','').str.replace('</b>','')
  for sen in frame['text']:
    if re.search('~',sen) != None:
      print(frame['Season'].unique(),sen)
      stoploop ='re'
    if re.search('\]',sen) != None or re.search('\[',sen) != None:
      print('\n',frame['Season'].unique(),sen)
      stoploop ='re'


 ['s02'] Nice, nice. Hey I got somethin' for you. [hands Chandler an envelope.

 ['s09'] Central Perk. Phoebe and Mike are on the sofa. Ross enters.]


In [18]:
# Find the brackets that are random / where there are action statements or noises included
# Remove either the bracket or the statement / noise
nums = ['0','1','2','3','4','5','6','7','8','9']

def checkbrackets(l):
  search = re.search('[)]',l)
  search1 = re.search('[(]',l)
  search3 = re.search('\[',l)

  nl = l
  # If there is one closing bracket
  if (search1 == None and search != None) or (search != None and search.span()[0]<search1.span()[0]):
    index = search.span()[0]

    # If there is a number in front of it, it is a summation
    if l[index-1] not in nums:
      nl = nl[:index]
      # Usually it is two punctuation marks before the bracket where it can be cut off
      for i in range(len(re.findall('[.?,]',nl))-1):
        insearch = re.search('[.?,]',nl).span()[1]
        nl = nl[insearch:]

      ind = re.search(nl[:7],l)
      newsen = l[:ind.span()[0]]

      # If there is text behind the bracket, include this
      if len(l[index+1:]) > 1:
        newsen = newsen + l[index+1:]

    else:
      newsen = nl

  elif search1 != None:
    index1 = search1.span()[0]
    nl = l[index1:]
    search2 = re.search('[)]',nl)
    newsen=nl

    if search2 != None:
      index2 = search2.span()[1]
      newsen=nl
      if index1 != 0:
        newsen = l[:index1-1]+l[index2+index1:]

    else:
      newsen = l[:index1]

  elif search3 != None:
    newsen = l[:search3.span()[0]]


  else:
    newsen = nl

  return newsen

In [19]:
# A few sentences were unique and had no applicable general rule
def hardcoding(sen):

  newsen = sen

  if 'Subconscious ' in sen:
    newsen = sen[:re.search('Subconscious',sen).span()[0] -11 ]+ sen[re.search('Subconscious',sen).span()[1] +1:]

  if 'leans back' in sen:
    newsen = sen[:re.search('leans back',sen).span()[0] ]

  if 'She walks' in sen:
    newsen = sen[:re.search('She walks',sen).span()[0]]

  if 'closes the laptop and joins Charlie on the sofa' in sen:
    newsen = 'Respectfully, professor R. Geller. Hey!'

  return newsen

In [20]:
%%time

friends = {}

# Check brackets because of weird occurances
for key in keys:
    frame = seasonal_frames[key]

    frame['text'] = frame.text.str.replace(' &#150','').str.replace('&ndash;','').str.replace('{','').str.replace('}','')

    frame['text'] = frame['text'].apply(lambda x : hardcoding(x))
    frame['text'] = frame['text'].apply(lambda x : checkbrackets(x))

    # else do not include sen
    for sen in frame['text']:
      if 'Central Perk. Phoebe and Mike are on the sofa. Ross enters.]' in sen or len(sen) < 2:
          df = frame[frame['text'] == sen]
          frame = frame.drop(df.index)

    if len(frame) > 10:
      friends[key] = frame

<timed exec>:7: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
<timed exec>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<timed exec>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

CPU times: user 3.23 s, sys: 84.1 ms, total: 3.31 s
Wall time: 3.29 s


In [21]:
frames = [friends[key] for key in list(friends.keys())]

In [22]:
%%time
# Check weird characters or letters

signs = []
for file in frames:
  n=-1
  for sen in file['text']:
    n+=1

    check = sen

    l = remnormals(sen)

    if re.search('[()]',sen) != None:
      sen = checkbrackets(sen)

    l = remnormals(sen)

    if len(sen) < 3:
      print(sen)
    # This way checked what strange letters and were.
    e = set(re.findall('[ÃÂâãËë]',l))
    # Larger than 1 because of ' '
    if len(e) > 1:
      print(' index',n, 'sentence: ',sen,'\n These are the characters: ',e,'\n')
      signs.append(e)

    # Other strange characters.
    e = set(re.findall('[^\w\*]',l))

    if len(e) > 1:
      print(' index',n,' sentence: ',sen,'\n These are the characters: ',e,'\n')
      signs.append(e)



Um
 index 273  sentence:  Oh, ahh, no thanks, I just had an M&M. 
 These are the characters:  {' ', '&'} 

 index 47  sentence:  You're into hardcore S&M right? 
 These are the characters:  {' ', '&'} 

E!
M!
I!
L!
Y!
 index 149  sentence:  Hi! Listen, can we watch cartoons on your television? We need a porn break. We spent the last two hours watching In & Out & In, Again. 
 These are the characters:  {' ', '&'} 

 index 201  sentence:  Okay. I have just a few questions to ask so I'm going to get out my official forms. Okay, so, question 1) You and uh, you were married to Francis' daughter Lilly, is that correct? 
 These are the characters:  {' ', ')'} 

 index 203  sentence:  Okay, umm, question 2) Umm, did that marriage end A. Happily, B. Medium, or C. In the total abandonment of her and her two children? 
 These are the characters:  {' ', ')'} 

 index 25  sentence:  I am so proud of Joey, I can't believe he's going to be on Law & Order! 
 These are the characters:  {' ', '&'} 

 in

In [ ]:
keys = list(friends.keys())
frames[0]

,conversation_id,text,speaker,Season,Episode,Season_Episode
0,s01_e01_c01_u001,There's nothing to tell! He's just some guy I ...,Monica Geller,s01,e01,"[s01, e01]"
1,s01_e01_c01_u001,"C'mon, you're going out with the guy! There's ...",Joey Tribbiani,s01,e01,"[s01, e01]"
2,s01_e01_c01_u001,"All right Joey, be nice. So does he have a hum...",Chandler Bing,s01,e01,"[s01, e01]"
3,s01_e01_c01_u001,"Wait, does he eat chalk?",Phoebe Buffay,s01,e01,"[s01, e01]"
5,s01_e01_c01_u001,"Just, 'cause, I don't want her to go through w...",Phoebe Buffay,s01,e01,"[s01, e01]"
...,...,...,...,...,...,...
318,s01_e01_c15_u001,"Yeah. Yeah, I'll have a cup of coffee.",#ALL#,s01,e01,"[s01, e01]"
319,s01_e01_c15_u001,"Kids, new dream... I'm in Las Vegas.",Chandler Bing,s01,e01,"[s01, e01]"
320,s01_e01_c15_u001,"Ahh, miss? More coffee?",Customer,s01,e01,"[s01, e01]"
321,s01_e01_c15_u001,"Ugh. Excuse me, could you give this to that gu...",Rachel Green,s01,e01,"[s01, e01]"


In [ ]:
%%time
friendsspeakers = {}
# Create the next speakers column
for key in keys:
  df = friends[key]
  next_speakers = [line for line in df.speaker]
  df['Next Speaker'] =  [next_speakers[0]] + next_speakers[:-1]
  friendsspeakers[key] = df

friendsspeakers[key]

CPU times: user 81.2 ms, sys: 654 µs, total: 81.9 ms
Wall time: 87.3 ms


,conversation_id,text,speaker,Season,Episode,Season_Episode,Next Speaker
67161,s10_e18_c01_u000,"Madame, your passport please?",Gate Attendant #1,s10,e18,"[s10, e18]",Gate Attendant #1
67162,s10_e18_c01_u000,Oh my God! I was so afraid I wasn't gonna reme...,Rachel Green,s10,e18,"[s10, e18]",Gate Attendant #1
67163,s10_e18_c01_u000,"Your boarding pass, please.",Gate Attendant #1,s10,e18,"[s10, e18]",Rachel Green
67164,s10_e18_c01_u000,Oh.,Rachel Green,s10,e18,"[s10, e18]",Gate Attendant #1
67165,s10_e18_c01_u000,"Oh, shoot. I had it. Oh, I can't believe this.",Rachel Green,s10,e18,"[s10, e18]",Rachel Green
...,...,...,...,...,...,...,...
67368,s10_e18_c11_u000,"Oh, it's gonna be okay.",Chandler Bing,s10,e18,"[s10, e18]",Monica Geller
67369,s10_e18_c11_u000,Do you guys have to go to the new house right ...,Rachel Green,s10,e18,"[s10, e18]",Chandler Bing
67370,s10_e18_c11_u000,We got some time.,Monica Geller,s10,e18,"[s10, e18]",Rachel Green
67371,s10_e18_c11_u000,"Okay, should we get some coffee?",Rachel Green,s10,e18,"[s10, e18]",Monica Geller


In [ ]:
# Store for later use/access
#with open('friendsspeakers.pkl', 'wb') as fp:
 #   pickle.dump(friendsspeakers, fp)
  #  print('dictionary saved successfully to file')

In [ ]:
# Load in previous dictionairy of dataframes, that need to be converterd to spark dataframes
with open('friendsspeakers.pkl', 'rb') as fp:
    dfs = pickle.load(fp)


In [ ]:
keys=list(dfs.keys())
dfs[keys[0]]

,conversation_id,text,speaker,Season,Episode,Season_Episode,Next Speaker
0,s01_e01_c01_u001,There's nothing to tell! He's just some guy I ...,Monica Geller,s01,e01,"[s01, e01]",Monica Geller
1,s01_e01_c01_u001,"C'mon, you're going out with the guy! There's ...",Joey Tribbiani,s01,e01,"[s01, e01]",Monica Geller
2,s01_e01_c01_u001,"All right Joey, be nice. So does he have a hum...",Chandler Bing,s01,e01,"[s01, e01]",Joey Tribbiani
3,s01_e01_c01_u001,"Wait, does he eat chalk?",Phoebe Buffay,s01,e01,"[s01, e01]",Chandler Bing
5,s01_e01_c01_u001,"Just, 'cause, I don't want her to go through w...",Phoebe Buffay,s01,e01,"[s01, e01]",Phoebe Buffay
...,...,...,...,...,...,...,...
318,s01_e01_c15_u001,"Yeah. Yeah, I'll have a cup of coffee.",#ALL#,s01,e01,"[s01, e01]",Rachel Green
319,s01_e01_c15_u001,"Kids, new dream... I'm in Las Vegas.",Chandler Bing,s01,e01,"[s01, e01]",#ALL#
320,s01_e01_c15_u001,"Ahh, miss? More coffee?",Customer,s01,e01,"[s01, e01]",Chandler Bing
321,s01_e01_c15_u001,"Ugh. Excuse me, could you give this to that gu...",Rachel Green,s01,e01,"[s01, e01]",Customer


## Spark frames

In [ ]:
!pip3 install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=e00be9ae597f3dc0ead7b4afa2c240568be0ea7d5c3aec44863a3b586d3b80ab
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import functions as F

In [ ]:
spark = SparkSession.builder.appName("Sparkify").config('spark.sql.session.timeZone', 'GMT+1') \
        .getOrCreate()

In [ ]:
%%time
keys = list(dfs.keys())
for k in keys:
  dfs[k] = spark.createDataFrame(dfs[k]).withColumn("Y", when(col("speaker") == col("Next Speaker"),0).otherwise(1))\
                .withColumn("Sentence Length",F.split('text',' ')).withColumn('Sentence Length',size('Sentence Length')).toPandas()

dfs[keys[0]]

CPU times: user 6.82 s, sys: 627 ms, total: 7.44 s
Wall time: 1min 23s


,conversation_id,text,speaker,Season,Episode,Season_Episode,Next Speaker,Y,Sentence Length
0,s01_e01_c01_u001,There's nothing to tell! He's just some guy I ...,Monica Geller,s01,e01,"[s01, e01]",Monica Geller,0,11
1,s01_e01_c01_u001,"C'mon, you're going out with the guy! There's ...",Joey Tribbiani,s01,e01,"[s01, e01]",Monica Geller,1,14
2,s01_e01_c01_u001,"All right Joey, be nice. So does he have a hum...",Chandler Bing,s01,e01,"[s01, e01]",Joey Tribbiani,1,16
3,s01_e01_c01_u001,"Wait, does he eat chalk?",Phoebe Buffay,s01,e01,"[s01, e01]",Chandler Bing,1,5
4,s01_e01_c01_u001,"Just, 'cause, I don't want her to go through w...",Phoebe Buffay,s01,e01,"[s01, e01]",Phoebe Buffay,0,16
...,...,...,...,...,...,...,...,...,...
295,s01_e01_c15_u001,"Yeah. Yeah, I'll have a cup of coffee.",#ALL#,s01,e01,"[s01, e01]",Rachel Green,1,8
296,s01_e01_c15_u001,"Kids, new dream... I'm in Las Vegas.",Chandler Bing,s01,e01,"[s01, e01]",#ALL#,1,7
297,s01_e01_c15_u001,"Ahh, miss? More coffee?",Customer,s01,e01,"[s01, e01]",Chandler Bing,1,4
298,s01_e01_c15_u001,"Ugh. Excuse me, could you give this to that gu...",Rachel Green,s01,e01,"[s01, e01]",Customer,1,20


In [ ]:
# Store for later use/access
#with open('friendssparks.pkl', 'wb') as fp:
#    pickle.dump(dfs, fp)
#    print('dictionary saved successfully to file')

dictionary saved successfully to file


In [ ]:
frames = [dfs[key] for key in keys]
frames[0]

,conversation_id,text,speaker,Season,Episode,Season_Episode,Next Speaker,Y,Sentence Length
0,s01_e01_c01_u001,There's nothing to tell! He's just some guy I ...,Monica Geller,s01,e01,"[s01, e01]",Monica Geller,0,11
1,s01_e01_c01_u001,"C'mon, you're going out with the guy! There's ...",Joey Tribbiani,s01,e01,"[s01, e01]",Monica Geller,1,14
2,s01_e01_c01_u001,"All right Joey, be nice. So does he have a hum...",Chandler Bing,s01,e01,"[s01, e01]",Joey Tribbiani,1,16
3,s01_e01_c01_u001,"Wait, does he eat chalk?",Phoebe Buffay,s01,e01,"[s01, e01]",Chandler Bing,1,5
4,s01_e01_c01_u001,"Just, 'cause, I don't want her to go through w...",Phoebe Buffay,s01,e01,"[s01, e01]",Phoebe Buffay,0,16
...,...,...,...,...,...,...,...,...,...
295,s01_e01_c15_u001,"Yeah. Yeah, I'll have a cup of coffee.",#ALL#,s01,e01,"[s01, e01]",Rachel Green,1,8
296,s01_e01_c15_u001,"Kids, new dream... I'm in Las Vegas.",Chandler Bing,s01,e01,"[s01, e01]",#ALL#,1,7
297,s01_e01_c15_u001,"Ahh, miss? More coffee?",Customer,s01,e01,"[s01, e01]",Chandler Bing,1,4
298,s01_e01_c15_u001,"Ugh. Excuse me, could you give this to that gu...",Rachel Green,s01,e01,"[s01, e01]",Customer,1,20


In [ ]:
# Create one concattenated frame of alle separate frames (subtitle files of series and movies)
%%time

concat_frame = pd.DataFrame()
for frame in frames:
  concat_frame = pd.concat([concat_frame,frame],ignore_index=True)

concat_frame

CPU times: user 641 ms, sys: 1.91 ms, total: 643 ms
Wall time: 649 ms


,conversation_id,text,speaker,Season,Episode,Season_Episode,Next Speaker,Y,Sentence Length
0,s01_e01_c01_u001,There's nothing to tell! He's just some guy I ...,Monica Geller,s01,e01,"[s01, e01]",Monica Geller,0,11
1,s01_e01_c01_u001,"C'mon, you're going out with the guy! There's ...",Joey Tribbiani,s01,e01,"[s01, e01]",Monica Geller,1,14
2,s01_e01_c01_u001,"All right Joey, be nice. So does he have a hum...",Chandler Bing,s01,e01,"[s01, e01]",Joey Tribbiani,1,16
3,s01_e01_c01_u001,"Wait, does he eat chalk?",Phoebe Buffay,s01,e01,"[s01, e01]",Chandler Bing,1,5
4,s01_e01_c01_u001,"Just, 'cause, I don't want her to go through w...",Phoebe Buffay,s01,e01,"[s01, e01]",Phoebe Buffay,0,16
...,...,...,...,...,...,...,...,...,...
61302,s10_e18_c11_u000,"Oh, it's gonna be okay.",Chandler Bing,s10,e18,"[s10, e18]",Monica Geller,1,5
61303,s10_e18_c11_u000,Do you guys have to go to the new house right ...,Rachel Green,s10,e18,"[s10, e18]",Chandler Bing,1,18
61304,s10_e18_c11_u000,We got some time.,Monica Geller,s10,e18,"[s10, e18]",Rachel Green,1,4
61305,s10_e18_c11_u000,"Okay, should we get some coffee?",Rachel Green,s10,e18,"[s10, e18]",Monica Geller,1,6


In [ ]:
#with open('friendsconcattedframe.pkl', 'wb') as fp:
#    pickle.dump(concat_frame, fp)
#    print('dictionary saved successfully to file')


dictionary saved successfully to file
